# Title - Fitbit Time Series Project

# Goal
1.


Notes for intial data peeking:
1. Missing data 6/27/2018 - 7/6/2018
2. Most of the records are missing Food/ Food log information.  
> Available: May - June 1 record, October - November 7 records
3. Overall data range 4/26/2018 - 12/6/2018

Index - 
April, May, June, Aug, Sept, Oct, Nov: 36-67
July: 36-66
Dec: 36-45

Hypothesis
1. Missing value can be predicted with by similar days of the weeks (weekday/ weekend
2. Missing value can be affected by month/ season of the year
3. Explore: hypothesis testing, df.corr on 
'cal_burn', 'steps', 'distance', 'floors', 'min_sed'
'min_sed','min_active_light', 'min_active_fairly', 'min_active_very','cal_activity'
'cal_burn', 'cal_activity'


In [21]:
df.columns

Index(['cal_burn', 'steps', 'distance', 'floors', 'min_sed',
       'min_active_light', 'min_active_fairly', 'min_active_very',
       'cal_activity'],
      dtype='object')

# Deliverables

1. Notebook: 
> material & methods, include data preparation, summary/ data dictionary
>
> 
>
> analyses
>
> conclusions
>
> predictions: 
>
> missing two weeks worth of data (csv file)
> 
> include comment about the individual who was wearing this fitness tracker

2. Prediction result: missing two weeks worth of data (csv file)

3. 2 slides, audience = a general audience. 
> Include at least one visualization, clearly labeled

In [1]:
# data manipulation 
import numpy as np
import pandas as pd

from datetime import datetime
import itertools as it

from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AR

from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics

import math

# data visualization 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# ignore warnings
import warnings
warnings.filterwarnings("ignore")
#import acquire
#import prepare
#import summarize
#from acquire import get_store_data
#from prepare import prep_store_data, split_store_data
#from summarize import df_summary

# Acquisition

In [2]:
df = pd.read_csv('act.csv')
df.head()

,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories
0,4/26/18,"2,635","2,931",1.38,2,"1,347",93,0,0,566
1,4/27/18,"3,649","9,971",4.70,19,"1,158",219,46,17,"1,752"
2,4/28/18,"3,379","3,521",1.66,2,702,155,22,11,"1,124"
3,4/29/18,"2,851","1,639",0.77,0,891,122,0,0,602
4,4/30/18,"2,825","1,323",0.62,0,880,104,0,0,540


In [3]:
df.columns

Index(['Date', 'Calories Burned', 'Steps', 'Distance', 'Floors',
       'Minutes Sedentary', 'Minutes Lightly Active', 'Minutes Fairly Active',
       'Minutes Very Active', 'Activity Calories'],
      dtype='object')

In [4]:
df.columns = ["date","cal_burn","steps","distance","floors","min_sed","min_active_light","min_active_fairly","min_active_very","cal_activity"]

In [5]:
str_attributes = df.select_dtypes(object).columns.to_list()

In [6]:
str_attributes = ['cal_burn', 'steps', 'min_sed', 'cal_activity']

In [7]:
def handle_obj_type(df):
    for s in str_attributes:
        df[s] = df[s].str.replace(',','')
        df[s] = df[s].astype('int')
    return df

In [8]:
df = handle_obj_type(df)

In [9]:
df.head()

,date,cal_burn,steps,distance,floors,min_sed,min_active_light,min_active_fairly,min_active_very,cal_activity
0,4/26/18,2635,2931,1.38,2,1347,93,0,0,566
1,4/27/18,3649,9971,4.70,19,1158,219,46,17,1752
2,4/28/18,3379,3521,1.66,2,702,155,22,11,1124
3,4/29/18,2851,1639,0.77,0,891,122,0,0,602
4,4/30/18,2825,1323,0.62,0,880,104,0,0,540


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 10 columns):
date                 225 non-null object
cal_burn             225 non-null int64
steps                225 non-null int64
distance             225 non-null float64
floors               225 non-null int64
min_sed              225 non-null int64
min_active_light     225 non-null int64
min_active_fairly    225 non-null int64
min_active_very      225 non-null int64
cal_activity         225 non-null int64
dtypes: float64(1), int64(8), object(1)
memory usage: 17.7+ KB


In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
cal_burn,225.0,3401.911111,632.426578,799.0,3069.00,3385.00,3765.0,5199.0
steps,225.0,8536.186667,5255.746608,0.0,4714.00,7806.00,12380.0,25512.0
distance,225.0,4.046800,2.506253,0.0,2.22,3.68,5.9,12.0
floors,225.0,12.591111,25.844174,0.0,2.00,7.00,13.0,195.0
min_sed,225.0,809.520000,243.232639,28.0,654.00,750.00,877.0,1440.0
min_active_light,225.0,188.235556,75.170001,0.0,158.00,203.00,236.0,349.0
min_active_fairly,225.0,14.920000,15.363535,0.0,3.00,11.00,21.0,78.0
min_active_very,225.0,33.871111,37.195986,0.0,2.00,15.00,63.0,154.0
cal_activity,225.0,1484.546667,747.795790,0.0,1062.00,1481.00,1962.0,3596.0


In [10]:
df.date = pd.to_datetime(df.date)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 10 columns):
date                 225 non-null datetime64[ns]
cal_burn             225 non-null int64
steps                225 non-null int64
distance             225 non-null float64
floors               225 non-null int64
min_sed              225 non-null int64
min_active_light     225 non-null int64
min_active_fairly    225 non-null int64
min_active_very      225 non-null int64
cal_activity         225 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(8)
memory usage: 17.7 KB


In [13]:
df = df.set_index('date')

In [15]:
df.head()

,cal_burn,steps,distance,floors,min_sed,min_active_light,min_active_fairly,min_active_very,cal_activity
date,,,,,,,,,
2018-04-26,2635,2931,1.38,2,1347,93,0,0,566
2018-04-27,3649,9971,4.70,19,1158,219,46,17,1752
2018-04-28,3379,3521,1.66,2,702,155,22,11,1124
2018-04-29,2851,1639,0.77,0,891,122,0,0,602
2018-04-30,2825,1323,0.62,0,880,104,0,0,540


In [19]:
print(len(df))
print(len(df.asfreq('D')))

225
225


In [20]:
df['2018-06']

,cal_burn,steps,distance,floors,min_sed,min_active_light,min_active_fairly,min_active_very,cal_activity
date,,,,,,,,,
2018-06-01,3513,6916,3.26,0,690,279,10,2,1527
2018-06-02,2742,2946,1.39,3,717,171,0,0,752
2018-06-03,3765,6815,3.21,2,545,201,42,29,1834
2018-06-04,3585,6162,2.91,6,1171,225,36,8,1530
2018-06-05,3189,4486,2.12,4,823,177,10,0,1022
2018-06-06,2145,0,0.00,0,1440,0,0,0,0
2018-06-07,2146,0,0.00,0,1440,0,0,0,0
2018-06-08,2465,1929,0.91,1,1361,79,0,0,389
2018-06-09,2859,3130,1.48,7,835,127,24,1,780


In [34]:
train = df['2018-07-16':]
test = df['2018-04':'2018-07-15']

In [35]:
test.tail()

,cal_burn,steps,distance,floors,min_sed,min_active_light,min_active_fairly,min_active_very,cal_activity
date,,,,,,,,,
2018-07-11,3553,6997,3.30,7,701,214,21,14,1495
2018-07-12,3750,7520,3.55,1,576,294,0,0,1787
2018-07-13,3641,7432,3.51,1,798,206,32,27,1627
2018-07-14,3622,11239,5.30,10,709,203,39,35,1770
2018-07-15,3315,6077,2.87,10,654,169,29,20,1354


In [36]:
train.head()

,cal_burn,steps,distance,floors,min_sed,min_active_light,min_active_fairly,min_active_very,cal_activity
date,,,,,,,,,
2018-07-16,4037,10551,4.98,4,714,204,51,47,2084
2018-07-17,3760,8377,3.95,12,646,258,8,25,1803
2018-07-18,3509,7504,3.54,4,754,222,20,3,1502
2018-07-19,3993,9484,4.47,4,1116,243,57,24,2093
2018-07-20,3110,6680,3.15,3,1262,136,28,14,1172


# Explore and Prepare

In [ ]:
# explore

In [ ]:
def prep_data(df: pd.DataFrame) -> pd.DataFrame:
    return (df.assign(ds=pd.to_datetime(df.Month_Invoiced)).sort_values('ds')
            .assign(y=df.Amount)\
            .groupby(['ds'])['y'].sum().reset_index().set_index('ds'))

In [ ]:
df = prep_data(df)

In [ ]:
def split_store_data(df, train_prop): 
    train_size = int(len(df) * train_prop)
    train, test = df[0:train_size].reset_index(), df[train_size:len(df)].reset_index()
    return train, test

In [ ]:
train, test = split_store_data(df2, train_prop=.80)

print('Observations: %d' % (len(df2)))
print('Training Observations: %d' % (len(train)))
print('Testing Observations: %d' % (len(test)))

In [ ]:
train = train.set_index('ds')
test = test.set_index('ds')

In [ ]:
# peek data
plt.figure(figsize=(12, 4))
plt.plot(train)
plt.plot(test)

# Model

In [ ]:
def evaluate(target_var, train = train, test = test, output=True):
    mse = metrics.mean_squared_error(test[target_var], yhat[target_var])
    rmse = math.sqrt(mse)

    if output:
        print('MSE:  {}'.format(mse))
        print('RMSE: {}'.format(rmse))
    else:
        return mse, rmse

def plot_and_eval(target_vars, train = train, test = test, metric_fmt = '{:.2f}', linewidth = 4):
    if type(target_vars) is not list:
        target_vars = [target_vars]

    plt.figure(figsize=(16, 8))
    plt.plot(train[target_vars],label='Train')
    plt.plot(test[target_vars], label='Test')

    for var in target_vars:
        mse, rmse = evaluate(target_var = var, train = train, test = test, output=False)
        plt.plot(yhat[var], linewidth=linewidth)
        print(f'{var} -- MSE: {metric_fmt} RMSE: {metric_fmt}'.format(mse, rmse))

    plt.show()

In [ ]:
eval_df = pd.DataFrame(columns=['model_type', 'target_var', 'metric', 'value'])
eval_df

In [ ]:
# create df to store MSE & RMSE
def append_eval_df(model_type, target_vars, train = train, test = test):
    temp_eval_df = pd.concat([pd.DataFrame([[model_type, i, 'mse', evaluate(target_var = i, 
                                                                            train = train, 
                                                                            test = test, 
                                                                            output=False)[0]],
                                            [model_type, i, 'rmse', evaluate(target_var = i, 
                                                                             train = train, 
                                                                             test = test, 
                                                                             output=False)[1]]],
                                           columns=['model_type', 'target_var', 'metric', 'value']) 
                              for i in target_vars], ignore_index=True)
    return eval_df.append(temp_eval_df, ignore_index=True)

In [ ]:
periods = 30
yhat['y'] = train.y.rolling(periods).mean().iloc[-1]

# plot_and_eval(target_vars, train, test)
# eval_df = append_eval_df(model_type='moving_average', target_vars=['y'], train = train, test = test)

plt.figure(figsize=(16, 8))
plt.plot(train[target_vars],label='Train')
plt.plot(test[target_vars], label='Test')
period_vals = [1, 4, 12, 26, 52, 104]
for p in period_vals:
    yhat['y'] = train.y.rolling(p).mean().iloc[-1]
    plt.plot(yhat.y)
    print('\nrolling averge period:',p)
    print('\nitem sales\n')
    evaluate('y', train = train, test = test)